In [ ]:
from glob import glob
import SimpleITK as sitk
import os
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from shutil import move
from itertools import product
import pandas as pd
from tqdm import tqdm_notebook
import nibabel as nib

In [ ]:
PATH = 'E:\\IDHwt_Perfusion_final\\SET_18'
in_path = PATH 
outpath = 'E:\\IDHwt_Perfusion_final\\Feature_extraction\\SET_18'
os.makedirs(outpath, exist_ok = True)
# maskDir = PATH + '00_Tumor_masks_resampled/'

project_name = 'Perfusion'

In [ ]:
from __future__ import print_function
import radiomics
from radiomics import featureextractor
import logging
radiomics.logger.setLevel(logging.ERROR)
import SimpleITK as sitk
from radiomics import firstorder, glcm, shape, glrlm, glszm, ngtdm, gldm
import numpy as np


def Numpy2Itk(array):
    '''
    :param array: numpy array format
    :return: simple itk image type format
    '''
    return sitk.GetImageFromArray(array)

def feature_extract(image_origin, image_mask, features = ['firstorder', 'glcm','gldm', 'glszm', 'glrlm', 'ngtdm', 'shape'], binWidth=32, binCount=None):
    '''
    :param image_origin: image_array (numpy array)
    :param image_mask: mask_array (numpy array)
    :return: whole features, featureVector
    '''
    image = Numpy2Itk(image_origin)
    mask = Numpy2Itk(image_mask)
    
    settings = {}

    if binWidth:
        settings['binWidth'] = binWidth
    if binCount:
        settings['binCount'] = binCount
    settings['resampledPixelSpacing'] = (1,1,1)
    settings['interpolator'] = 'sitkBSpline'
    settings['verbose'] = True
    
    extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
    extractor.settings['enableCExtensions'] = True
    
    for feature in features:
        extractor.enableFeatureClassByName(feature.lower())
        
    featureVector = extractor.execute(image, mask)
    
    cols = []; feats = []
    for feature in features:
        for featureName in sorted(featureVector.keys()):
            if feature in featureName:
                cols.append(featureName)
                feats.append(featureVector[featureName])
    return feats, cols

In [ ]:
for (path, dirs, files) in os.walk(in_path):
    print(path, dirs, files)

In [ ]:
pt_lst = os.listdir(in_path)
pt_lst = sorted([int(pt) for pt in pt_lst if 'txt' not in pt and 'xlsx' not in pt and 'txt' not in pt])
pt_lst = [str(pt) for pt in pt_lst]
pt_lst, len(pt_lst)

In [ ]:
error_dict = {}
for size in [8, 32, 128]:
    feat_df_width = pd.DataFrame()
    feat_df_count = pd.DataFrame()
    total = 0
    for (path, dirs, files) in os.walk(in_path):
        if dirs == []:
            if 'output' in os.path.basename(path):
                total += 1
                pt = path.split(os.path.sep)[-2]
                
                if 'dsc' in os.path.basename(path):
                    seq = 'dsc'
                    
                elif 'dce' in os.path.basename(path):
                    seq = 'dce'
                    
                else:
                    print("Something wrong....")
                    print(path)
                    continue
                
                print("# {} : Feature Extraction in {} / {}".format(total, pt, seq))
                
                # ==== Brain path 설정 / 순서 맞춰주려고 정렬시킴
                brain_paths = [os.path.join(path,file) for file in sorted(files) if 'nii' in file]
                
                if seq == 'dsc':
                    if len(brain_paths) != 5:
                        print("Something wrong in DSC...")
                        print([file for file in sorted(files) if 'nii' in file])
                        print()
                        continue
                        
                elif seq == 'dce':
                    if len(brain_paths) != 7:
                        print("Something wrong in DCE...")
                        print([file for file in sorted(files) if 'nii' in file])
                        print()
                        continue
                
                print("Brain_paths: ")
                print([file for file in sorted(files) if 'nii' in file])
                print()
                
                # ==== Mask path 설정
                mask_paths = sorted(glob(os.path.join(in_path, pt, '*mask_label*')))
                
                if len(mask_paths) == 0:
                    print("tumor mask not exists")
                    print()
                    continue
                
                print("Mask_paths: ")
                print([os.path.basename(mask) for mask in mask_paths])
                print()
                

        bw_features_lst, bw_cols_lst, bc_features_lst, bc_cols_lst = [], [], [], []
    
# ------------------------
        
        for (img_path, mask_path) in product(brain_paths, mask_paths):
            try:
                print(img_path, mask_path)
                mask_type = '_'.join(os.path.basename(mask_path).split("_")[2:4])

                brain_img, mask = [sitk.ReadImage(path) for path in [img_path, mask_path]]
                brain_arr, mask_arr = [sitk.GetArrayFromImage(img) for img in [brain_img, mask]]

# -----------------------
                # Scale
#                brain_arr = brain_arr * 100

                # Change binWidth
                bw_features, bw_cols = feature_extract(brain_arr, mask_arr, binWidth=size)
                bw_cols = [col.replace('original', '{}'.format(seq)) for col in bw_cols]
#                bw_cols = [col.replace('original', '{}_image_{}_mask'.format(seq, mask_type)) for col in bw_cols]

                # Change binCount            
                bc_features, bc_cols = feature_extract(brain_arr, mask_arr, binWidth=None, binCount=size)
                bc_cols = [col.replace('original', '{}'.format(seq)) for col in bc_cols]
#                bc_cols = [col.replace('original', '{}_image_{}_mask'.format(seq, mask_type)) for col in bc_cols]

                bw_features_lst += bw_features
                bw_cols_lst += bw_cols
                bc_features_lst += bc_features
                bc_cols_lst += bc_cols

            except Exception as e:
                print("error", e, path)
                error_dict[path] = e
                
        feat_df_width = pd.concat([feat_df_width, pd.DataFrame(np.array(bw_features_lst),index=bw_cols_lst, columns=[pt]).T])
        feat_df_count = pd.concat([feat_df_count, pd.DataFrame(np.array(bc_features_lst), index=bc_cols_lst, columns=[pt]).T])

    feat_df_width.to_csv(os.path.join(outpath, '{}_bin_width_{}.csv'.format(project_name, size)))
    feat_df_count.to_csv(os.path.join(outpath, '{}_bin_count_{}.csv'.format(project_name, size)))
    print("Finished %d size" % size)
    print()
    print()
    print()